In [1]:
import concurrent.futures
import pathlib
import os

import polars as pl
import plotnine as p9

from tools.sequence.sequence import Sequence

In [2]:
df = pl.read_csv(pathlib.Path(os.getcwd()) / ".." / "_analysis_data" / "output" / "1-optimised.csv")
df

source,name,input_sequence,output_sequence,failed_reason
str,str,str,str,str
"""hrt_hk_nt_seq_pulled_from_ense…","""ENSG00000004059|ENSG0000000405…","""ATGGGCCTCACCGTGTCCGCGCTCTTTTCG…","""ATGGGCCTGACCGTGAGCGCCCTGTTCAGC…",""""""
"""hrt_hk_nt_seq_pulled_from_ense…","""ENSG00000003056|ENSG0000000305…","""ATGTTCCCTTTCTACAGCTGCTGGAGGACT…","""ATGTTCCCCTTCTACAGCTGCTGGAGAACC…",""""""
"""hrt_hk_nt_seq_pulled_from_ense…","""ENSG00000006451|ENSG0000000645…","""ATGGCTGCAAATAAGCCCAAGGGTCAGAAT…","""ATGGCCGCCAACAAGCCCAAGGGCCAGAAC…",""""""
"""hrt_hk_nt_seq_pulled_from_ense…","""ENSG00000007520|ENSG0000000752…","""ATGGGCCGCAGGAGGGCAGCGCGCGGGCCG…","""ATGGGAAGAAGAAGAGCCGCCAGAGGCCCC…","""CAI < 0.9"""
"""hrt_hk_nt_seq_pulled_from_ense…","""ENSG00000004975|ENSG0000000497…","""ATGGCGGGTAGCAGCACTGGGGGCGGTGGG…","""ATGGCCGGATCTAGCACAGGCGGCGGAGGC…",""""""
…,…,…,…,…
"""ensembl_ccds_tx_nt_seq.fa""","""ENSG00000173846|ENSG0000017384…","""ATGGAGCCTGCCGCCGGTTTCCTGTCTCCG…","""""","""GC content constraint not reso…"
"""ensembl_ccds_tx_nt_seq.fa""","""ENSG00000142166|ENSG0000014216…","""ATGGATAATTGGATAAAATTGTCTGGGTGT…","""ATGGACAACTGGATCAAGCTGAGCGGCTGC…",""""""
"""ensembl_ccds_tx_nt_seq.fa""","""ENSG00000180488|ENSG0000018048…","""ATGTCAGACTGCTGCTCAGCGCCAGGCATC…","""ATGAGCGACTGCTGCTCTGCCCCTGGCATT…",""""""


In [4]:
def _mfe(sequence: str) -> float | None:
    if not sequence:
        return None

    return Sequence.from_string(sequence).minimum_free_energy()

with concurrent.futures.ProcessPoolExecutor() as executor:
    results = executor.map(_mfe, df["input_sequence"].to_list())
    df["input_mfe"] = pl.Series(results)

with concurrent.futures.ProcessPoolExecutor() as executor:
    results = executor.map(_mfe, df["output_sequence"].to_list())
    df["output_mfe"] = pl.Series(results)

df.write_csv(output_dir / "2.1-hydrated-mfe.csv")

TypeError: 'MinimumFreeEnergy' object is not callable